In [1]:
import main as m
# reload
import importlib as i
# reload if it is neccesarry i.reload(m)

In [10]:
PATH = {}
PATH["weather"] = "data/weather/"
DS = m.xr.open_dataset(PATH["weather"]+"orig_CO_2019_ym.nc")

# write to csv
# DS.to_dataframe().to_csv("output_filename.csv")
# read in
df = DS.to_dataframe()
df = m.gpd.GeoDataFrame(
    df, geometry=m.gpd.points_from_xy(df.lon, df.lat))

# convert values
df["lon"] = df["lon"].astype(float)
df["lat"] = df["lat"].astype(float)
# double conversation needs
df.CO = df.CO.astype(float).astype(int)
# important! 
df.crs = "EPSG:23700" #Hungary
#print(df.head())
# filter, since there are duplications
df = df[df["Times_bnds"] == 20190101.0]
###########################################
###########################################
# filter focusing on Hungary
df = df[df["lon"] < 23.02]
df = df[df["lon"] > 16.072]
df = df[df["lat"] > 45.699]
df = df[df["lat"] < 48.7]

# since there are only points, expand them to square
for key in df.index:
    df.geometry[key] = m.Polygon(((df.lon[key]-0.1, df.lat[key]),
                (df.lon[key], df.lat[key]),
               (df.lon[key], df.lat[key]+0.1),
           
               (df.lon[key]-0.1, df.lat[key]+0.1)))

### Filtering

A simple filter for CO level and map areas

In [11]:
print("Choose between ", min(df.CO), " ", max(df.CO)) 
COLEVELEXCEED = int(input())
#print(COLEVELEXCEED)
# set a mask for developing reasons
df.crs = "epsg:4326"
dfF = df[df.CO > COLEVELEXCEED]
#dfF = df[df.CO > 150]

# get boudaries
## boundaries
boundary1 = m.gpd.GeoSeries(m.unary_union(dfF.geometry))
# plot
m.multiPolygonVisualizer(boundary1)

Choose between  145   434
160


In [14]:
print(df.head())
df.explore("CO", cmap="YlGn")

                                             Times_bnds   lon   lat   CO  \
lev Times        bnds south_north west_east                                
1.0 2.019070e+07 0    7           21         20190101.0  16.1  45.7  178   
                                  22         20190101.0  16.2  45.7  173   
                                  23         20190101.0  16.3  45.7  168   
                                  24         20190101.0  16.4  45.7  167   
                                  25         20190101.0  16.5  45.7  165   

                                                                                      geometry  
lev Times        bnds south_north west_east                                                     
1.0 2.019070e+07 0    7           21         POLYGON ((16.00000 45.70000, 16.10000 45.70000...  
                                  22         POLYGON ((16.10000 45.70000, 16.20000 45.70000...  
                                  23         POLYGON ((16.20000 45.70000, 16.30

In [ ]:
def airQualityAverager(AIRQUALITYFILELIST, MEASURE):
    return df